# Validations

## Anycast IPs previous to target validation

Check if the ips previous to target for every target if the are identified as anycast

In [ ]:
import ipinfo
import pandas as pd
from ast import literal_eval

from src.utils.common_functions import (
    json_file_to_dict,
    dict_to_json_file
)
from src.utils.constants import (
    KEYS_FILEPATH,
    REPLICATION_PACKAGE_DIR,
    RESULTS_MODES
)

In [ ]:
ANALYSIS_MODE=RESULTS_MODES[0]

ANALYSIS_FOLDER = f"{REPLICATION_PACKAGE_DIR}/analysis_{ANALYSIS_MODE}"
ROUTES_RESULTS_FILENAME = f"{ANALYSIS_FOLDER}/routes_results_{ANALYSIS_MODE}.csv"
IPS_PREVIOUS_TO_TARGET_CLASSIFIED_FILENAME = f"{REPLICATION_PACKAGE_DIR}/{ANALYSIS_FOLDER}/ips_previous_to_target_classified_{ANALYSIS_MODE}.json"

In [ ]:
def is_ip_anycast_request(ip_to_check: str):
    access_token = json_file_to_dict(KEYS_FILEPATH)["ipinfo_token"]
    handler = ipinfo.getHandler(access_token)
    details = handler.getDetails(ip_to_check)
    #print(json.dumps(details.all, indent=4))
        
    try:
        anycast = details.anycast
    except:
        anycast = False
        
    return anycast

In [ ]:
routes_df = pd.read_csv(ROUTES_RESULTS_FILENAME)
ips_previous_to_target = set()
for index, row in routes_df.iterrows():
    ips_route = set(literal_eval(routes_df["ips_previous_to_target"]))
    ips_previous_to_target.update(ips_route)

try:
    ips_previous_to_target_classified: dict[str:bool] = json_file_to_dict(IPS_PREVIOUS_TO_TARGET_CLASSIFIED_FILENAME)
except:
    ips_previous_to_target_classified: dict[str:bool] = {}
    
for ip in ips_previous_to_target:
    if ip in ips_previous_to_target_classified.keys():
        continue
    is_anycast = is_ip_anycast_request(ip)
    ips_previous_to_target_classified[ip] = is_anycast
    
    dict_to_json_file(
        ips_previous_to_target_classified,
        IPS_PREVIOUS_TO_TARGET_CLASSIFIED_FILENAME
    )
